In [134]:
import pandas as pd
import numpy as np
import math
# from data.get_data import run_query, install_backlog_sql, maint_backlog_sql, live_fleet_sql, new_sales_sla_sql, locs_in_implementation_sql

In [135]:
input_data = 'binder TEMPLATE Tech Hiring.xlsx'

In [136]:
vendor_capacity = pd.read_excel(input_data, sheet_name='vendor inputs',).iloc[:2, :]
vendor_capacity = vendor_capacity.set_index('name')
vendor_cohort = pd.read_excel(input_data, sheet_name='vendor inputs', header=5, usecols="A:D").iloc[:12, :]
vendor_cohort_st = pd.read_excel(input_data, sheet_name='vendor inputs', header=23, usecols="A:C")
vendor_cohort_st = vendor_cohort_st.set_index('Service Territory')

In [137]:
# vendor_cohort.pivot(index='Service Territory', columns='Wo Created', values='Ct').reset_index()
nsa_df = vendor_cohort[vendor_cohort['vendor']=='NSA']
dish_df = vendor_cohort[vendor_cohort['vendor']=='DISH']

nsa_df = nsa_df.pivot_table(index=['cohort', 'WO Type'], values=['capacity', 'WO Type'], aggfunc={'capacity': np.sum}).reset_index()
nsa_df.columns = ['NSA cohort', 'WO Type', 'capacity']

dish_df = dish_df.pivot_table(index=['cohort', 'WO Type'], values=['capacity', 'WO Type'], aggfunc={'capacity': np.sum}).reset_index()
dish_df.columns = ['Dish cohort', 'WO Type', 'capacity']

nsa_df

,NSA cohort,WO Type,capacity
0,a,M,500.0
1,b,M,500.0
2,c,M,500.0
3,d,M,400.0
4,e,M,400.0
5,f,M,400.0


In [138]:
nsa_vendor_cohort_st = vendor_cohort_st.reset_index().merge(nsa_df, how='left', on='NSA cohort').set_index('Service Territory')
dish_vendor_cohort_st = vendor_cohort_st.reset_index().merge(dish_df, how='left', on='Dish cohort').set_index('Service Territory')

nsa_vendor_cohort_st

,NSA cohort,Dish cohort,WO Type,capacity
Service Territory,,,,
Alabama - South,a,a,M,500.0
Alabama Central,a,a,M,500.0
"Albany, GA",a,a,M,500.0
"Albany, NY",a,a,M,500.0
"Albuquerque, NM",a,a,M,500.0
...,...,...,...,...
Washington D.C.,f,f,M,400.0
West - Unassigned,f,f,M,400.0
"West LA, CA",f,f,M,400.0


### Main Service Territory

In [139]:
st_index = vendor_cohort_st.index
st_df = pd.DataFrame(index=st_index)
st_df.head()

""
Service Territory
Alabama - South
Alabama Central
"Albany, GA"
"Albany, NY"
"Albuquerque, NM"


In [140]:
def align_st(df):
    res_df = pd.concat([st_df, df], axis=1)
    res_df = st_df.merge(df, how='left', on='Service Territory')
    return res_df

def convert_list_date_cols_to_str(df):
    col_list = list(df.columns)
    res_list = [str(x)[:10] for x in col_list]

    return res_list

def convert_date_idx_to_str(df_ser):
    idx_list = list(df_ser.index)
    idx_str_list = [str(x)[:10] for x in idx_list]
    df_ser.index = idx_str_list 

    return df_ser

def get_rolling_df(df):
    res_df = df.cumsum(axis=1)
    return res_df

### SQL inputs

In [141]:
# install_backlog_df = run_query(install_backlog_sql)
# install_backlog_df = install_backlog_df.set_index('service_territory')
# install_backlog_df.index.names = ['Service Territory']

# maint_backlog_df = run_query(maint_backlog_sql)
# maint_backlog_df = maint_backlog_df.set_index('service_territory')
# maint_backlog_df.index.names = ['Service Territory']

# live_fleet_df = run_query(live_fleet_sql)
# live_fleet_df = live_fleet_df.set_index('service_territory')
# live_fleet_df.index.names = ['Service Territory']
# live_fleet_df = align_st(live_fleet_df)
# live_fleet_df = live_fleet_df.fillna(0)

# locs_in_implementation_df = run_query(locs_in_implementation_sql)
# locs_in_implementation_df = locs_in_implementation_df.set_index('service_territory')
# locs_in_implementation_df.index.names = ['Service Territory']


In [142]:
install_backlog_df = pd.read_excel(input_data, sheet_name='installs',)
install_backlog_df = install_backlog_df.set_index('Service Territory')
install_backlog_df.columns = ['age_month', 'ct']

locs_in_implementation_df = pd.read_excel(input_data, sheet_name='implementation',)
locs_in_implementation_df = locs_in_implementation_df.set_index('Service Territory')
locs_in_implementation_df.columns = ['age_month', 'ct']

live_fleet_df = pd.read_excel(input_data, sheet_name='live_fleet',)
live_fleet_df = live_fleet_df.set_index('Service Territory')
live_fleet_df.columns = ['live_fleet_cnt', 'ttl_fleet', 'live_fleet_perc']
live_fleet_df = align_st(live_fleet_df)
live_fleet_df = live_fleet_df.fillna(0)

### Template inputs

In [143]:
single_inputs = pd.read_excel(input_data, sheet_name='single inputs',)
single_inputs = single_inputs.set_index('Type')

monthly_inputs = pd.read_excel(input_data, sheet_name='monthly inputs',).iloc[:4, :]
monthly_inputs = monthly_inputs.set_index('name')

winter_maint_mom = pd.read_excel(input_data, sheet_name='monthly inputs',header=8,)
winter_maint_mom = winter_maint_mom.set_index('Service Territory')
winter_maint_mom = align_st(winter_maint_mom)
winter_maint_mom = winter_maint_mom.fillna(0)
winter_maint_mom.columns = convert_list_date_cols_to_str(winter_maint_mom)

sales_funnel_sla_df = pd.read_excel(input_data, sheet_name='sales_funnel', header=2).iloc[:4, :]
sales_funnel_sla_df = sales_funnel_sla_df.set_index('WO created')

new_sales_sla = pd.read_excel(input_data, sheet_name='sales_funnel', header=8, usecols="A:B",).iloc[:1, :]
new_sales_sla = new_sales_sla.set_index('Type')

sales_distb = pd.read_excel(input_data, sheet_name='sales_funnel', header=11,).iloc[:, :]
sales_distb = sales_distb.set_index('Service Territory')
sales_distb = align_st(sales_distb)
sales_distb = sales_distb.fillna(0)

initial_tech_count = pd.read_excel(input_data, sheet_name='initial_tech_count',)
initial_tech_count = initial_tech_count.set_index('Service Territory')
initial_tech_count = align_st(initial_tech_count)
initial_tech_count = initial_tech_count.fillna(0)

initial_local_tech_count=initial_tech_count[['1P tech count']]
initial_travel_tech_count=initial_tech_count[['Travel tech count']]

max_local_tech_hires = pd.read_excel(input_data, sheet_name='local_tech_hires',).fillna(0)
max_local_tech_hires = max_local_tech_hires.set_index('Service Territory').iloc[0]
max_local_tech_hires = convert_date_idx_to_str(max_local_tech_hires)

local_tech_hires = pd.read_excel(input_data, sheet_name='local_tech_hires',).iloc[1:, :]
local_tech_hires = local_tech_hires.set_index('Service Territory')
local_tech_hires = align_st(local_tech_hires)
local_tech_hires.columns = convert_list_date_cols_to_str(local_tech_hires)
local_tech_hires = local_tech_hires.fillna(0)

#### Single Inputs

In [144]:
num_wks_in_month = 4.33
flight_tech_df = single_inputs

num_months_val = flight_tech_df.loc['Number of months', 'Value']
install_with_ss_perc_val = flight_tech_df.loc['Installs w/ SS perc', 'Value']
ss_perc_3p_val = flight_tech_df.loc['SS perc 3P', 'Value']

new_sales_sla_val = new_sales_sla.loc['New Sales SLA (months)', 'Value']


### Main cols

In [145]:
main_col_list = convert_list_date_cols_to_str(monthly_inputs)
main_col_list

['2024-08-01',
 '2024-09-01',
 '2024-10-01',
 '2024-11-01',
 '2024-12-01',
 '2025-01-01',
 '2025-02-01',
 '2025-03-01',
 '2025-04-01',
 '2025-05-01',
 '2025-06-01',
 '2025-07-01']

### Tech monthly runrate less site surveys

In [146]:
vendor_install_budget_cap = convert_date_idx_to_str(vendor_capacity.loc['3P install max (budget)'])
vendor_maint_budget_cap = convert_date_idx_to_str(vendor_capacity.loc['3P maint max (budget)'])

In [147]:
wo_tech_wkyl_rr = monthly_inputs.loc['WO / tech / week', :]

wo_tech_wkyl_rr_less_ss = wo_tech_wkyl_rr * (1-(install_with_ss_perc_val*(1-ss_perc_3p_val)))
wo_tech_wkyl_rr_less_ss = wo_tech_wkyl_rr_less_ss.apply(lambda x: math.ceil(x))

wo_tech_mnthly_rr_less_ss = wo_tech_wkyl_rr_less_ss * num_wks_in_month
wo_tech_mnthly_rr_less_ss = wo_tech_mnthly_rr_less_ss.apply(lambda x: math.ceil(x))

wo_tech_mnthly_rr_less_ss = convert_date_idx_to_str(wo_tech_mnthly_rr_less_ss)
wo_tech_mnthly_rr_less_ss

2024-08-01    61
2024-09-01    61
2024-10-01    61
2024-11-01    61
2024-12-01    61
2025-01-01    61
2025-02-01    61
2025-03-01    65
2025-04-01    65
2025-05-01    65
2025-06-01    65
2025-07-01    65
Name: WO / tech / week, dtype: int64

### Perc capacity towards installs

In [148]:
install_perc_cap_input = monthly_inputs.loc['perc capacity towards installs (3P)']
install_perc_cap_input = convert_date_idx_to_str(install_perc_cap_input)

install_perc_cap_input

2024-08-01    0.9
2024-09-01    0.9
2024-10-01    0.9
2024-11-01    0.9
2024-12-01    0.9
2025-01-01    0.9
2025-02-01    0.9
2025-03-01    0.9
2025-04-01    0.9
2025-05-01    0.9
2025-06-01    0.9
2025-07-01    0.9
Name: perc capacity towards installs (3P), dtype: object

### initial local & travel tech capacity

### Install & Implementation Backlog

#### implementation wo backlog piv

In [149]:

locs_in_implementation_df_c = locs_in_implementation_df.copy()
implementation_backlog_gb_wo = locs_in_implementation_df_c.groupby(['Service Territory', 'age_month']).sum('ct').reset_index()

implementation_wo_backlog_piv = implementation_backlog_gb_wo.pivot(index='Service Territory', columns='age_month', values='ct').reset_index()
implementation_wo_backlog_piv = implementation_wo_backlog_piv.set_index('Service Territory')
implementation_wo_backlog_piv = align_st(implementation_wo_backlog_piv)
implementation_wo_backlog_piv.fillna(0, inplace=True)

implementation_wo_backlog_piv.head()

,2021-06-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,...,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01
Service Territory,,,,,,,,,,,,,,,,,,,,,
Alabama - South,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alabama Central,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,153.0,0.0,5.0,6.0,0.0,0.0,2.0,0.0
"Albany, GA",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,66.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
"Albany, NY",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Albuquerque, NM",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


#### install wo backlog piv

In [150]:
install_backlog_sql_c = install_backlog_df.copy()
install_backlog_gb_wo = install_backlog_sql_c.groupby(['Service Territory', 'age_month']).sum('ct').reset_index()

install_wo_backlog_piv = install_backlog_gb_wo.pivot(index='Service Territory', columns='age_month', values='ct').reset_index()
install_wo_backlog_piv = install_wo_backlog_piv.set_index('Service Territory')
install_wo_backlog_piv = align_st(install_wo_backlog_piv)
install_wo_backlog_piv.fillna(0, inplace=True)

# do we need to set the date range in sql

# install_wo_backlog_piv = install_wo_backlog_piv[[c for c in install_wo_backlog_piv.columns if c in main_col_list]]

install_wo_backlog_piv

,2023-09-01 00:00:00,2023-10-01 00:00:00,2023-11-01 00:00:00,2023-12-01 00:00:00,2024-01-01 00:00:00,2024-02-01 00:00:00,2024-03-01 00:00:00,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00
Service Territory,,,,,,,,,,,,,
Alabama - South,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alabama Central,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0
"Albany, GA",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,0.0
"Albany, NY",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,40.0,0.0
"Albuquerque, NM",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,11.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Washington D.C.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,3.0,35.0,1.0
West - Unassigned,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"West LA, CA",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,14.0,59.0,202.0,0.0


#### install wo backlog piv + implementation

In [151]:
install_wo_backlog_piv = pd.concat([install_wo_backlog_piv, implementation_wo_backlog_piv]).groupby('Service Territory').sum().reset_index()
install_wo_backlog_piv = install_wo_backlog_piv.set_index('Service Territory')

install_wo_backlog_col_list = list(install_wo_backlog_piv.columns)
install_wo_backlog_col_list = [str(x)[:10] for x in install_wo_backlog_col_list]
install_wo_backlog_piv.columns = install_wo_backlog_col_list
install_wo_backlog_piv = install_wo_backlog_piv.groupby(install_wo_backlog_piv.columns, axis=1).sum()

install_wo_backlog_piv

,2021-06-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,...,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01
Service Territory,,,,,,,,,,,,,,,,,,,,,
Alabama - South,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alabama Central,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,153.0,0.0,24.0,6.0,0.0,0.0,2.0,0.0
"Albany, GA",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,66.0,0.0,48.0,4.0,0.0,0.0,0.0,0.0
"Albany, NY",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,1.0,40.0,0.0,0.0,0.0,0.0,0.0
"Albuquerque, NM",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,0.0,1.0,12.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Washington D.C.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,9.0,4.0,3.0,38.0,6.0,0.0,0.0,0.0,0.0
West - Unassigned,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"West LA, CA",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,...,3.0,18.0,20.0,75.0,238.0,9.0,28.0,0.0,0.0,0.0


### Sales Data

In [152]:
sales_dt = sales_funnel_sla_df.copy()
sales_dt = sales_dt.sum()

sales_funnel_distb_c = sales_distb.copy()

sales_funnel_sla = sales_funnel_distb_c * sales_dt
sales_funnel_sla_col_list = list(sales_funnel_sla.columns)
sales_funnel_sla_col_list = [str(x)[:10] for x in sales_funnel_sla_col_list]

sales_funnel_sla.columns = sales_funnel_sla_col_list

sales_funnel_sla = sales_funnel_sla[main_col_list].fillna(0)
sales_funnel_sla = sales_funnel_sla.applymap(lambda x: x if x%.5!=0 else x+.1)
sales_funnel_sla = sales_funnel_sla.round(0) # rounds odd unit values up, and even unit values down for .5

sales_funnel_sla

# sales_funnel_distb_c
# sales_dt

,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01
Service Territory,,,,,,,,,,,,
Alabama - South,0.0,0.0,14.0,13.0,12.0,14.0,15.0,17.0,18.0,16.0,13.0,12.0
Alabama Central,0.0,0.0,27.0,27.0,25.0,27.0,30.0,34.0,36.0,32.0,27.0,24.0
"Albany, GA",0.0,0.0,56.0,55.0,51.0,57.0,62.0,71.0,75.0,65.0,55.0,49.0
"Albany, NY",0.0,0.0,77.0,76.0,69.0,77.0,85.0,97.0,103.0,90.0,76.0,67.0
"Albuquerque, NM",0.0,0.0,63.0,62.0,56.0,63.0,69.0,79.0,83.0,73.0,62.0,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...
Washington D.C.,0.0,0.0,142.0,140.0,128.0,143.0,157.0,179.0,189.0,165.0,140.0,124.0
West - Unassigned,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"West LA, CA",0.0,0.0,136.0,134.0,122.0,137.0,151.0,172.0,182.0,159.0,134.0,119.0


#### sales funnel sla

### Unconstrained Install dt MoM by ST

In [153]:
backlog_date = '2024-07-01'
install_dt_unconstrained_full = pd.concat([sales_funnel_sla, install_wo_backlog_piv]).reset_index().groupby('Service Territory', sort=False).sum(min_count=1)
install_dt_unconstrained_full.sort_index(axis=1, inplace=True)

install_dt_unconstrained_full_cum = install_dt_unconstrained_full.cumsum(axis=1)
install_backlog_ser = install_dt_unconstrained_full_cum[backlog_date]

install_dt_unconstrained = install_dt_unconstrained_full.loc[:,main_col_list[0]:main_col_list[-1]]
install_dt_unconstrained_w_backlog = pd.concat([install_backlog_ser, install_dt_unconstrained],  axis=1)

roll_install_dt_unconstrained = get_rolling_df(install_dt_unconstrained_w_backlog).iloc[:, 1:]

install_dt_unconstrained.head(12)

,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01
Service Territory,,,,,,,,,,,,
Alabama - South,0.0,0.0,14.0,13.0,12.0,14.0,15.0,17.0,18.0,16.0,13.0,12.0
Alabama Central,24.0,6.0,27.0,27.0,27.0,27.0,30.0,34.0,36.0,32.0,27.0,24.0
"Albany, GA",48.0,4.0,56.0,55.0,51.0,57.0,62.0,71.0,75.0,65.0,55.0,49.0
"Albany, NY",40.0,0.0,77.0,76.0,69.0,77.0,85.0,97.0,103.0,90.0,76.0,67.0
"Albuquerque, NM",12.0,0.0,63.0,62.0,56.0,63.0,69.0,79.0,83.0,73.0,62.0,55.0
Atlanta,169.0,12.0,178.0,172.0,157.0,176.0,194.0,220.0,233.0,203.0,172.0,153.0
"Austin, TX",19.0,4.0,26.0,25.0,23.0,26.0,28.0,32.0,34.0,30.0,25.0,22.0
"Birmingham, AL",25.0,0.0,34.0,21.0,23.0,20.0,22.0,25.0,27.0,23.0,20.0,18.0
"Boston, MA",18.0,0.0,30.0,30.0,38.0,30.0,34.0,38.0,40.0,35.0,30.0,27.0


### install dt backlog

### Maintenance

In [154]:
maint_creation_df = monthly_inputs.loc['maint creation % of fleet',:]
maint_creation_df.columns = convert_date_idx_to_str(maint_creation_df)

init_live_fleet = float(live_fleet_df['ttl_fleet'].iloc[0])
init_st_mnthly_live_fleet_perc = live_fleet_df['live_fleet_perc'].astype(float)

maint_creation_df

2024-08-01    0.1
2024-09-01    0.1
2024-10-01    0.1
2024-11-01    0.1
2024-12-01    0.1
2025-01-01    0.1
2025-02-01    0.1
2025-03-01    0.1
2025-04-01    0.1
2025-05-01    0.1
2025-06-01    0.1
2025-07-01    0.1
Name: maint creation % of fleet, dtype: object

### Unconstrained Maint dt MoM by ST

### Functions

In [155]:
import warnings
warnings.filterwarnings("ignore")

def get_current_month_local_tech_supply(qtrly_tech_count_ser, rem_inst_ser, monthly_tech_rr, dt_col, itr):
    
    cur_mon_local_tech_supply_ser_list = []
    for serv_tr, v in rem_inst_ser.items():

        rem_inst_ser = rem_inst_ser.astype(float)
        st_demand = rem_inst_ser.get(key = serv_tr)

        st_qtrly_tech_count = qtrly_tech_count_ser[qtrly_tech_count_ser.index == serv_tr][0]
        st_qtrly_tech_cap = st_qtrly_tech_count * monthly_tech_rr
        st_qtrly_tech_cap = st_qtrly_tech_cap * install_perc_cap_input.loc[dt_col]
        
        st_qtrly_tech_cap =  st_qtrly_tech_cap if st_qtrly_tech_cap%.5!=0 else st_qtrly_tech_cap + .1

        st_qtrly_tech_cap = round(st_qtrly_tech_cap)       

        st_cur_mon_local_tech_supply = st_qtrly_tech_cap if st_demand >= st_qtrly_tech_cap else st_demand
        
        cur_mon_local_tech_supply_ser_list.append((serv_tr, st_cur_mon_local_tech_supply)) 

    idx, values = zip(*cur_mon_local_tech_supply_ser_list) 
    cur_mon_local_tech_supply_ser = pd.Series(values, idx) # column i4
    cur_mon_local_tech_supply_ser.rename(dt_col, inplace=True)

    return cur_mon_local_tech_supply_ser

def round_down(num, divisor):
    return math.floor(num / divisor) * divisor

def techs_needed_to_complete_work(current_month_install_dt_ser, cur_mon_local_tech_supply_ser,monthly_tech_rr, itr, max_travel_tech_cap=None):
    cur_mon_techs_need_ser = (current_month_install_dt_ser - cur_mon_local_tech_supply_ser)/ monthly_tech_rr
    cur_mon_techs_need_ser.fillna(0, inplace=True)

    cur_mon_techs_need_ser = cur_mon_techs_need_ser.apply(lambda x: round_down(x, .05))

    if max_travel_tech_cap:
        cur_mon_techs_need_ser = cur_mon_techs_need_ser.apply(lambda x: min(max_travel_tech_cap, x))
    
    cur_mon_techs_need_ser.rename_axis("Service Territory", inplace=True)
    cur_mon_techs_need_ser = cur_mon_techs_need_ser.astype(float)

    cur_mon_techs_need_ser_ranked = cur_mon_techs_need_ser.rank(method='first', ascending=False)
    cur_mon_techs_need_ser_rank_x_sort = cur_mon_techs_need_ser_ranked.sort_values()

    return cur_mon_techs_need_ser, 'delete cur_mon_techs_need_ser_rank_x_sort'

def constrain_tech_needs(cur_mon_excess_tech_needs_ser, st_ttl_travel_tech_cnt_ser, monthly_tech_rr, dt_col):

    cur_mon_excess_tech_needs_cum_sum_ser = cur_mon_excess_tech_needs_ser.sort_values(ascending=False).cumsum()

    cur_mon_excess_tech_needs_df = pd.concat([cur_mon_excess_tech_needs_ser, cur_mon_excess_tech_needs_cum_sum_ser], axis=1)
    cur_mon_excess_tech_needs_df.columns = ['need', 'cum_sum']

    monthly_flight_cap = st_ttl_travel_tech_cnt_ser.sum() * monthly_tech_rr
    monthly_flight_cap = round(monthly_flight_cap * install_perc_cap_input[dt_col]) # rounds odd unit values up, and even unit values down for .5

    num_travel_techs = round(monthly_flight_cap/monthly_tech_rr) # rounds odd unit values up, and even unit values down for .5

    cur_mon_excess_tech_needs_df['need_delta'] = num_travel_techs - cur_mon_excess_tech_needs_df['cum_sum']

    cur_mon_excess_tech_needs_df['overage'] = cur_mon_excess_tech_needs_df['need'] + cur_mon_excess_tech_needs_df['need_delta']
    cur_mon_excess_tech_needs_df['overage'] = cur_mon_excess_tech_needs_df['overage'].apply(lambda x: max(x, 0))

    cur_mon_excess_tech_needs_df['final'] = \
        np.where(cur_mon_excess_tech_needs_df['need_delta'] > 0, cur_mon_excess_tech_needs_df['need'], cur_mon_excess_tech_needs_df['overage'])

    cur_mon_travel_tech_supply_ser = cur_mon_excess_tech_needs_df['final'].apply(lambda x: math.floor(x * monthly_tech_rr))
    cur_mon_travel_tech_supply_ser.rename(dt_col, inplace=True)

    return cur_mon_travel_tech_supply_ser

def constrain_tech_needs_maint(cur_mon_excess_tech_needs_ser, temp_list, monthly_tech_rr, dt_col, itr):
    ser_cum_sum = cur_mon_excess_tech_needs_ser.sort_values(ascending=False).cumsum()
    ser_df = pd.concat([cur_mon_excess_tech_needs_ser, ser_cum_sum], axis=1)
    ser_df.columns = ['val', 'cum_sum']

    ser_df['delta'] = float(temp_list) - ser_df['cum_sum']
    ser_df['overage'] = ser_df['val'] + ser_df['delta']

    ser_df['overage'] = ser_df['overage'].apply(lambda x: max(x, 0))

    ser_df['final'] = np.where(ser_df['delta'] > 0, ser_df['val'], ser_df['overage'])

    cur_mon_travel_tech_supply_ser = ser_df['final']
    cur_mon_travel_tech_supply_ser = cur_mon_travel_tech_supply_ser.sort_values()
    cur_mon_travel_tech_supply_ser = cur_mon_travel_tech_supply_ser.apply(lambda x: math.floor(x * monthly_tech_rr))

    cur_mon_travel_tech_supply_ser.rename(dt_col, inplace=True)

    return cur_mon_travel_tech_supply_ser

def get_remaining_install_dt(remaining_install_dt_ser, current_month_local_tech_supply_ser, current_month_travel_tech_supply_ser, current_month_install_dt_ser, dt_col):

    df_test = pd.concat([remaining_install_dt_ser, current_month_local_tech_supply_ser, current_month_travel_tech_supply_ser, current_month_install_dt_ser], axis=1)
    df_test.columns = ['remaining_install_dt', 'current_month_local_tech_supply', 'current_month_travel_tech_supply', 'current_month_install_dt']

    df_test['labor_supply'] = df_test['current_month_local_tech_supply'] + df_test['current_month_travel_tech_supply']
    df_test['surplus_install_dt'] = df_test['remaining_install_dt'] - df_test['labor_supply'] + df_test['current_month_install_dt']
    df_test['surplus_install_dt'] = df_test['surplus_install_dt'].apply(lambda x: max(0,x))

    surplus_install_dt_ser = df_test['surplus_install_dt']
    surplus_install_dt_ser.rename(dt_col,inplace=True)

    return surplus_install_dt_ser

def get_new_tech_hire_list(surplus_install_dt_ser, max_hires_val, st_overwrite_ser, monthly_tech_rr):
    # remove st with values greater than zero in hiring table and reduce max_hires_val, then add back in at the end
    st_overwrites_list = list(st_overwrite_ser.index.unique()) # st's to remove
    surplus_install_dt_ser = surplus_install_dt_ser[~surplus_install_dt_ser.index.isin(st_overwrites_list)] # remove st from this ser

    techs_for_remaining_installs_ser = surplus_install_dt_ser / monthly_tech_rr
    techs_for_remaining_installs_ser.fillna(0, inplace=True)
    techs_for_remaining_installs_ser = techs_for_remaining_installs_ser.apply(math.floor)

    remaining_installs_rank = surplus_install_dt_ser.rank(method='first', ascending=False)
    techs_for_remaining_installs_ser = techs_for_remaining_installs_ser.apply(lambda x: min(x, 3))
    remaining_installs_rank.sort_values(inplace=True)

    df_aa = pd.concat([techs_for_remaining_installs_ser, remaining_installs_rank], axis=1)
    df_aa.columns = ['tech', 'rank']

    df_aa.sort_values(by=['rank'], inplace=True)
    df_aa['cum_sum'] = df_aa['tech'].cumsum()
    # df_aa['net_adds_list_itr'] = net_adds_list[net_adds_list_itr]
    # df_aa['net_adds_delta'] = net_adds_list[net_adds_list_itr] - df_aa['cum_sum']
    df_aa['net_adds_delta'] = max_hires_val - df_aa['cum_sum']
    df_aa['overage'] = df_aa['tech'] + df_aa['net_adds_delta']
    df_aa['overage'] = df_aa['overage'].apply(lambda x: max(x, 0))

    df_aa['final'] = np.where(df_aa['net_adds_delta'] > 0, df_aa['tech'], df_aa['overage'])

    res_ser = df_aa['final'].combine(st_overwrite_ser, max, fill_value=0)
    
    return res_ser

def update_tech_cnt_by_cap_ser(tech_cnt_by_cap_ser_c, local_tech_hire_ser, monthly_tech_rr, dt_col):
        
    sum_df = pd.concat([tech_cnt_by_cap_ser_c, local_tech_hire_ser], axis=1)
    sum_df.fillna(0, inplace=True)
    sum_df.columns = ['tech_cnt_by_cap_ser_col', 'final']
    sum_df['mnthly_wo'] = sum_df['final'] * monthly_tech_rr
    # sum_df['adds'] = sum_df['tech_cnt_by_cap_ser_col'] + sum_df['mnthly_wo']
    sum_df['adds'] = sum_df['tech_cnt_by_cap_ser_col'] + sum_df['final'] # was monthly_wo
    
    sum_df[dt_col] = sum_df['tech_cnt_by_cap_ser_col'] + sum_df['final'] # was monthly_wo
    # tech_cnt_by_cap_ser_c_list.append(sum_df[dt_col])
    
    new_tech_cnt_by_cap_ser = sum_df['adds']

    return new_tech_cnt_by_cap_ser



In [156]:
def get_w384(new_list_ser, ser_b3, h102_ser, itr):
    w384_list = []

    if itr > 0:

        maint_aged_dt_roll_ser = maint_dt_unconstrained.iloc[:,itr]

        for i, v in new_list_ser.items():
            maint_aged_dt_roll_val = maint_aged_dt_roll_ser[maint_aged_dt_roll_ser.index == i][0]
            ser_b3_val = ser_b3[ser_b3.index == i][0]
            h102_ser_val = h102_ser[h102_ser.index == i][0]

            inner_value = (v + maint_aged_dt_roll_val)-(ser_b3_val + h102_ser_val)
            max_val = max(inner_value, 0)
            w384_list.append((i, max_val))

        idx, values = zip(*w384_list) 
        w384_ser = pd.Series(values, idx)  

    else:
        for i, v in new_list_ser.items():
            
            max_val=max(v-(ser_b3[ser_b3.index == i][0] + h102_ser[h102_ser.index == i][0]),0)

            w384_list.append((i, max_val))

        idx, values = zip(*w384_list) 
        w384_ser = pd.Series(values, idx) 

    w384_ser = w384_ser.astype(float)

    return w384_ser

def get_b384(w384_ser, itr, dt_col):

    tartget_3p_maint = 1400
    b384_list = []
    func_itr=0
    for i, v in w384_ser.items():
        if func_itr == 0:
            ser_sum_pct = v/w384_ser.sum()
            
            round_up_sum = math.ceil(ser_sum_pct*100)/100 # round up 2
            min_target_v_tot = min(tartget_3p_maint, w384_ser.sum())
            multiply_vals = round_up_sum * min_target_v_tot
            round_mul_val = round(multiply_vals)
            final_val = min(round_mul_val, v)

        else:
            tuple_vals = [x[1] for x in b384_list]
            running_sum = sum(tuple_vals)
            ser_sum_pct = v/w384_ser.sum()
  
            round_up_sum = math.ceil(ser_sum_pct*100)/100 # round up 2
            min_target_v_tot = min(tartget_3p_maint, w384_ser.sum())
            multiply_vals = round_up_sum * min_target_v_tot
            round_down = math.floor(multiply_vals*100)/100

            min_left = min(round_down,w384_ser.sum())
            left_val = running_sum + min_left

            if left_val > min_target_v_tot:
                delta = min_target_v_tot - running_sum
                minum = min(delta, v)
                final_val = max(minum, 0)
            
            else:
                rsum_times_targ = round_up_sum*min_target_v_tot
                rdown = math.floor(rsum_times_targ*100)/100
                final_val = min(rdown, v)


        func_itr =+ 1
        b384_list.append((i, final_val))

    idx, values = zip(*b384_list) 
    b384_ser = pd.Series(values, idx) 

    b384_ser = b384_ser.astype(float)
    b384_ser.rename(dt_col, inplace=True)

    return b384_ser


In [157]:
def capacity_constraint(in_df, needed_capcity, max_capacity, res_name):
    df = in_df.copy()
    df['cumsum'] = df[needed_capcity].cumsum()
    df['delta'] = df[max_capacity] - df['cumsum']
    df['overage'] = df[needed_capcity] + df['delta']
    df['overage'] = df['overage'].apply(lambda x: max(x, 0))

    df[res_name] = np.where(df['delta'] > 0, df[needed_capcity], df['overage'])
    df = df[[res_name]]

    res_df = pd.concat([in_df, df], axis=1)
    return res_df

def constrain_external_tech_needs(cur_mon_excess_external_tech_needs_ser, vendor_cohort_st, vendor, monthly_tech_rr, dt_col, rem_max_val=None):
    all_vendor_df = pd.concat([cur_mon_excess_external_tech_needs_ser, vendor_cohort_st], axis=1)

    cohort_list = list(all_vendor_df[vendor].unique())

    cohort_df_list = []
    for cohort in cohort_list:
        
        cohort_df = all_vendor_df[all_vendor_df[vendor]==cohort]

        cohort_df = cohort_df[[0, 'capacity']].sort_values(by=[0], ascending=False)
        cohort_df['needed_cap'] = cohort_df[0] * monthly_tech_rr
        cohort_df['needed_cap'] = cohort_df['needed_cap'].apply(lambda x: round(x)) # change to regular round later
    
        cohort_df = capacity_constraint(cohort_df, 'needed_cap', 'capacity', 'cohort_capacity')

        # cohort_df = cohort_df[cohort_df['cohort_capacity'] > 0]
        cohort_df = cohort_df[['cohort_capacity']].reset_index()

        cohort_df['cohort'] = cohort

        cohort_df_list.append(cohort_df)

    all_cohort_df = pd.concat(cohort_df_list, ignore_index=True, sort=False)
    all_cohort_df = all_cohort_df.sort_values(by=['cohort_capacity'], ascending=False)

    if rem_max_val is not None:
        max_capacity_val = vendor_maint_budget_cap.loc[dt_col] - rem_max_val
    else:
        max_capacity_val = vendor_install_budget_cap.loc[dt_col]

    all_cohort_df['max_capacity'] = max_capacity_val

    all_cohort_df = capacity_constraint(all_cohort_df, 'cohort_capacity', 'max_capacity', dt_col)

    all_cohort_df[dt_col] = all_cohort_df[dt_col]

    all_cohort_df = all_cohort_df.set_index('Service Territory')

    return all_cohort_df[[dt_col]]

def monthly_maint(live_fleet, live_fleet_st_perc, dt_col):
    perc_maint_of_live_fleet = maint_creation_df.loc[dt_col]
    ttl_main_vol = live_fleet * perc_maint_of_live_fleet
    st_mnthly_maint_dt = np.outer(live_fleet_st_perc, ttl_main_vol)

    st_mnthly_maint_dt = pd.DataFrame(index=live_fleet_st_perc.index, data=st_mnthly_maint_dt, columns=[dt_col])

    st_mnthly_maint_dt[dt_col] = st_mnthly_maint_dt[dt_col] + winter_maint_mom[dt_col]

    st_mnthly_maint_dt = st_mnthly_maint_dt.applymap(lambda x: x if x%.5!=0 else x+.1)
    st_mnthly_maint_dt = st_mnthly_maint_dt.round(0) # rounds odd unit values up, and even unit values down for .5

    return st_mnthly_maint_dt

### Install Logic

In [158]:
remaining_install_dt_ser_list = []
tech_cap_by_st_ser_list = []
travel_tech_cap_by_st_ser_list = []
local_tech_install_supply_list = []
travel_tech_install_supply_list = []
local_tech_hire_list = []
as_install_list = []

internal_tech_needs_list = []
external_tech_needs_list = []
internal_tech_supply_list = []
external_tech_supply_list = []

net_adds_list = [14, 16, 13, 13, 26, 28]

# this is only ran once
remaining_install_dt_ser_list.append(install_backlog_ser) 

tech_cnt_by_cap_ser_c = initial_local_tech_count.copy()
tech_cnt_by_cap_ser_c_ = initial_local_tech_count['1P tech count']
tech_cnt_by_cap_ser_c_.name = main_col_list[0]


st_travel_tech_cnt_ser = initial_travel_tech_count.copy()

tech_hiring_itr = 0
net_adds_list_itr = 0

for itr in range(0, len(main_col_list)):

    dt_col = main_col_list[itr]

    remaining_install_dt_ser = remaining_install_dt_ser_list[itr]
    remaining_install_dt_ser.fillna(0, inplace=True)

    current_month_install_dt_ser = install_dt_unconstrained.loc[:, dt_col] # only used in last call

    # Need to get new runrate every month
    monthly_tech_rr = wo_tech_mnthly_rr_less_ss.loc[dt_col]

    # if itr !=0 and itr % 3 == 0:
    cur_max_hires_val = max_local_tech_hires[itr]
    if cur_max_hires_val > 0:

        local_tech_hire_ser = local_tech_hire_list[tech_hiring_itr]
        new_tech_cnt_by_cap_ser = update_tech_cnt_by_cap_ser(tech_cnt_by_cap_ser_c, local_tech_hire_ser, monthly_tech_rr, dt_col)
        
        tech_cnt_by_cap_ser_c = new_tech_cnt_by_cap_ser
        tech_cnt_by_cap_ser_c.rename('1P tech count', inplace=True) # new
        tech_cnt_by_cap_ser_c = tech_cnt_by_cap_ser_c.to_frame() # new

        tech_hiring_itr += 1

    st_grounded_travel_tech_cnt_ser = st_travel_tech_cnt_ser.astype(float).applymap(lambda x: math.ceil(x * .5))

    st_ttl_local_tech_cnt_ser = tech_cnt_by_cap_ser_c['1P tech count'] + st_grounded_travel_tech_cnt_ser['Travel tech count']
    st_ttl_travel_tech_cnt_ser = st_travel_tech_cnt_ser['Travel tech count'] - st_grounded_travel_tech_cnt_ser['Travel tech count']

    st_ttl_local_tech_cnt_ser.name = dt_col
    tech_cap_by_st_ser_list.append(st_ttl_local_tech_cnt_ser)

    current_month_local_tech_supply_ser = get_current_month_local_tech_supply(st_ttl_local_tech_cnt_ser, remaining_install_dt_ser, monthly_tech_rr, dt_col, itr)
    local_tech_install_supply_list.append(current_month_local_tech_supply_ser) 

    cur_mon_excess_tech_needs_ser, _ = techs_needed_to_complete_work(remaining_install_dt_ser, current_month_local_tech_supply_ser, monthly_tech_rr, itr)
    internal_tech_needs_list.append(cur_mon_excess_tech_needs_ser)
    
    current_month_travel_tech_supply_ser = constrain_tech_needs(cur_mon_excess_tech_needs_ser, st_ttl_travel_tech_cnt_ser, monthly_tech_rr, dt_col)
    travel_tech_install_supply_list.append(current_month_travel_tech_supply_ser)

    surplus_install_dt_ser = get_remaining_install_dt(remaining_install_dt_ser, current_month_local_tech_supply_ser, current_month_travel_tech_supply_ser, current_month_install_dt_ser, dt_col)

    if itr < len(main_col_list)-1:
        next_month_max_hires_val = max_local_tech_hires[itr + 1]

    if next_month_max_hires_val > 0 and itr < len(main_col_list)-1:
        nxt_dt_col = main_col_list[itr + 1]
        st_overwrite_ser = local_tech_hires[local_tech_hires[nxt_dt_col] > 0][nxt_dt_col]

        next_month_max_hires_val = next_month_max_hires_val - st_overwrite_ser.sum()
        new_tech_hiring_ser = get_new_tech_hire_list(surplus_install_dt_ser, next_month_max_hires_val, st_overwrite_ser, monthly_tech_rr)

        new_tech_hiring_ser.name = dt_col
        local_tech_hire_list.append(new_tech_hiring_ser)
        net_adds_list_itr += 1


    current_month_internal_tech_supply_ser = current_month_local_tech_supply_ser + current_month_travel_tech_supply_ser
    internal_tech_supply_list.append(current_month_internal_tech_supply_ser)


    cur_mon_excess_external_tech_needs_ser, _ = techs_needed_to_complete_work(remaining_install_dt_ser, current_month_internal_tech_supply_ser, monthly_tech_rr, itr)
    external_tech_needs_list.append(cur_mon_excess_external_tech_needs_ser)

    current_external_tech_supply_ser = constrain_external_tech_needs(cur_mon_excess_external_tech_needs_ser, dish_vendor_cohort_st,'Dish cohort' , monthly_tech_rr, dt_col)
    external_tech_supply_list.append(current_external_tech_supply_ser)
    
    surplus_install_dt_ser = get_remaining_install_dt(remaining_install_dt_ser, current_month_internal_tech_supply_ser, current_external_tech_supply_ser, current_month_install_dt_ser, dt_col)
    
    remaining_install_dt_ser_list.append(surplus_install_dt_ser)
    
    as_ser = current_month_internal_tech_supply_ser + current_external_tech_supply_ser[dt_col]

    as_install_list.append(as_ser)

local_tech_supply = pd.concat(local_tech_install_supply_list, axis=1)
travel_tech_supply = pd.concat(travel_tech_install_supply_list, axis=1)
intall_df = pd.concat(remaining_install_dt_ser_list, axis=1)
qtrly_tech_cap = pd.concat(tech_cap_by_st_ser_list, axis=1)
met_install_df = pd.concat(as_install_list, axis=1)

internal_tech_needs_df = pd.concat(internal_tech_needs_list, axis=1) # after local supply
external_tech_needs_df = pd.concat(external_tech_needs_list, axis=1)
internal_tech_supply_df = pd.concat(internal_tech_supply_list, axis=1)
external_tech_supply_df = pd.concat(external_tech_supply_list, axis=1)



In [159]:
qtrly_tech_cap.sum()

2024-08-01    129.0
2024-09-01    129.0
2024-10-01    135.0
2024-11-01    135.0
2024-12-01    135.0
2025-01-01    149.0
2025-02-01    149.0
2025-03-01    163.0
2025-04-01    163.0
2025-05-01    177.0
2025-06-01    177.0
2025-07-01    177.0
dtype: float64

In [160]:
roll_num = 3
cap_perc_ma3 = install_dt_unconstrained.rolling(roll_num, axis=1).mean() / (qtrly_tech_cap.rolling(3, axis=1).mean() * wo_tech_mnthly_rr_less_ss.rolling(3, axis=0).mean())
cap_perc_ma3 = cap_perc_ma3.iloc[:, roll_num-1:]
cap_perc_ma3 = cap_perc_ma3.replace([np.inf, -np.inf], 0).fillna(0)
cap_perc_ma3

,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01
Service Territory,,,,,,,,,,
Alabama - South,0.076503,0.147541,0.213115,0.213115,0.224044,0.245989,0.261780,0.261538,0.241026,0.210256
Alabama Central,0.311475,0.327869,0.442623,0.442623,0.459016,0.486631,0.523560,0.523077,0.487179,0.425641
"Albany, GA",0.590164,0.628415,0.885246,0.890710,0.928962,1.016043,1.089005,1.082051,1.000000,0.866667
"Albany, NY",0.639344,0.836066,1.213115,1.213115,1.262295,1.385027,1.492147,1.487179,1.379487,1.194872
"Albuquerque, NM",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
Washington D.C.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
West - Unassigned,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"West LA, CA",0.449180,0.335519,0.459016,0.429508,0.448087,0.491979,0.528796,0.526154,0.487179,0.422564


In [161]:
cap_perc = install_dt_unconstrained / (qtrly_tech_cap * wo_tech_mnthly_rr_less_ss)
cap_perc = cap_perc.replace([np.inf, -np.inf], 0).fillna(0)
cap_perc

,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01
Service Territory,,,,,,,,,,,,
Alabama - South,0.000000,0.000000,0.229508,0.213115,0.196721,0.229508,0.245902,0.261538,0.276923,0.246154,0.200000,0.184615
Alabama Central,0.393443,0.098361,0.442623,0.442623,0.442623,0.442623,0.491803,0.523077,0.553846,0.492308,0.415385,0.369231
"Albany, GA",0.786885,0.065574,0.918033,0.901639,0.836066,0.934426,1.016393,1.092308,1.153846,1.000000,0.846154,0.753846
"Albany, NY",0.655738,0.000000,1.262295,1.245902,1.131148,1.262295,1.393443,1.492308,1.584615,1.384615,1.169231,1.030769
"Albuquerque, NM",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
Washington D.C.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
West - Unassigned,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"West LA, CA",0.780328,0.029508,0.537705,0.439344,0.400000,0.449180,0.495082,0.529231,0.560000,0.489231,0.412308,0.366154


In [162]:
(qtrly_tech_cap * wo_tech_mnthly_rr_less_ss)

,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01
Alabama - South,61,61,61.0,61.0,61.0,61.0,61.0,65.0,65.0,65.0,65.0,65.0
Alabama Central,61,61,61.0,61.0,61.0,61.0,61.0,65.0,65.0,65.0,65.0,65.0
"Albany, GA",61,61,61.0,61.0,61.0,61.0,61.0,65.0,65.0,65.0,65.0,65.0
"Albany, NY",61,61,61.0,61.0,61.0,61.0,61.0,65.0,65.0,65.0,65.0,65.0
"Albuquerque, NM",0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
Washington D.C.,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
West - Unassigned,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"West LA, CA",305,305,305.0,305.0,305.0,305.0,305.0,325.0,325.0,325.0,325.0,325.0
"Wichita, KS",0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [163]:
def remaining_vendor_cap(external_tech_supply_df, vendor_cohort):
    df = pd.concat([vendor_cohort['Dish cohort'], external_tech_supply_df], axis=1)
    cohort_list = list(vendor_cohort['Dish cohort'].unique())
    df_cohort_list = []
    for cohort in cohort_list:
        dish_cohort_cap_val = vendor_cohort[vendor_cohort['Dish cohort']=='a']['capacity'].iloc[0]
        df_res = dish_cohort_cap_val - df[df['Dish cohort']==cohort].sum().iloc[1:]
        df_res.name = 'capacity'
        df_res = df_res.to_frame()
        df_res['cohort'] = cohort
        df_cohort_list.append(df_res.reset_index())
    
    all_cohort_df = pd.concat(df_cohort_list, ignore_index=True, sort=False)
    return all_cohort_df


def format_vendor_cohort_df(external_tech_supply_df, vendor_cohort, dt_col):
    df_mom = remaining_vendor_cap(external_tech_supply_df, vendor_cohort)
    df_mom = df_mom.set_index('index').loc[dt_col]

    vendor_cohort = vendor_cohort.iloc[:,:-1].reset_index()
    df_res = vendor_cohort.merge(df_mom, left_on='Dish cohort', right_on='cohort', how='left').set_index('Service Territory')
    df_res = df_res[list(df_res.columns[:-1])]
    return df_res


dt_col = main_col_list[0]
format_vendor_cohort_df(external_tech_supply_df, dish_vendor_cohort_st, dt_col)

,NSA cohort,Dish cohort,WO Type,capacity
Service Territory,,,,
Alabama - South,a,a,B,467.0
Alabama Central,a,a,B,467.0
"Albany, GA",a,a,B,467.0
"Albany, NY",a,a,B,467.0
"Albuquerque, NM",a,a,B,467.0
...,...,...,...,...
Washington D.C.,f,f,B,394.0
West - Unassigned,f,f,B,394.0
"West LA, CA",f,f,B,394.0


### Maint. logic

In [164]:
intial_maint_dt_ser = (live_fleet_df['live_fleet_cnt'] * .1).apply(lambda x: round(x))
intial_maint_dt_ser.name = backlog_date
intial_maint_dt_ser

Service Territory
Alabama - South       32
Alabama Central       46
Albany, GA            91
Albany, NY            39
Albuquerque, NM       23
                    ... 
Washington D.C.       79
West - Unassigned      1
West LA, CA          206
Wichita, KS           75
Wyoming                2
Name: 2024-07-01, Length: 91, dtype: int64

In [165]:
met_install_df.index.names = ['Service Territory']
rolling_maint_dt_unconstrained = live_fleet_df[['live_fleet_cnt']].merge(met_install_df, on='Service Territory', how='left').cumsum(axis=1).iloc[:,1:]
maint_dt_unconstrained_list = []

for idx, val in maint_creation_df.items():
    ser = rolling_maint_dt_unconstrained[idx]*val
    ser = ser.apply(lambda x: math.ceil(x))
    maint_dt_unconstrained_list.append(ser)
    

maint_dt_unconstrained = pd.concat(maint_dt_unconstrained_list, axis=1)
maint_dt_unconstrained

,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01
Service Territory,,,,,,,,,,,,
Alabama - South,32,32,32,34,35,36,38,39,41,43,44,46
Alabama Central,63,65,66,68,71,74,77,80,83,87,90,92
"Albany, GA",99,104,105,110,116,121,126,133,140,147,154,159
"Albany, NY",50,54,54,62,69,76,84,92,102,112,121,129
"Albuquerque, NM",24,25,25,32,38,43,49,57,64,73,80,86
...,...,...,...,...,...,...,...,...,...,...,...,...
Washington D.C.,83,87,87,101,115,128,142,158,176,195,211,226
West - Unassigned,1,1,1,1,1,1,1,1,1,1,1,1
"West LA, CA",221,245,246,262,275,288,301,316,334,352,368,381


In [166]:
maint_dt_unconstrained_w_backlog = pd.concat([intial_maint_dt_ser, maint_dt_unconstrained], axis=1)
maint_dt_unconstrained_w_backlog

,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01
Service Territory,,,,,,,,,,,,,
Alabama - South,32,32,32,32,34,35,36,38,39,41,43,44,46
Alabama Central,46,63,65,66,68,71,74,77,80,83,87,90,92
"Albany, GA",91,99,104,105,110,116,121,126,133,140,147,154,159
"Albany, NY",39,50,54,54,62,69,76,84,92,102,112,121,129
"Albuquerque, NM",23,24,25,25,32,38,43,49,57,64,73,80,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Washington D.C.,79,83,87,87,101,115,128,142,158,176,195,211,226
West - Unassigned,1,1,1,1,1,1,1,1,1,1,1,1,1
"West LA, CA",206,221,245,246,262,275,288,301,316,334,352,368,381


In [167]:
maint_dt_unconstrained_ = maint_dt_unconstrained.copy()
maint_dt_unconstrained_['2024-08-01'] = maint_dt_unconstrained_['2024-08-01'] + maint_dt_unconstrained_w_backlog['2024-07-01']
maint_dt_unconstrained_.head()

,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01
Service Territory,,,,,,,,,,,,
Alabama - South,64,32,32,34,35,36,38,39,41,43,44,46
Alabama Central,109,65,66,68,71,74,77,80,83,87,90,92
"Albany, GA",190,104,105,110,116,121,126,133,140,147,154,159
"Albany, NY",89,54,54,62,69,76,84,92,102,112,121,129
"Albuquerque, NM",47,25,25,32,38,43,49,57,64,73,80,86


In [168]:
remaining_maint_dt_ser_list = []
as_maint_list = []

local_tech_maint_supply_list = []
travel_tech_maint_supply_list = []
internal_maint_tech_supply_list = []
external_maint_tech_supply_list = []
nsa_external_maint_tech_supply_list = []
dish_external_maint_tech_supply_list = []

external_maint_tech_needs_list = []
external_maint_post_nsa_tech_needs_list = []

remaining_maint_dt_ser_list.append(intial_maint_dt_ser)

max_travel_tech_perc = 1  # user input

for itr in range(0, len(main_col_list)):
    dt_col = main_col_list[itr]

    remaining_maint_dt_ser = remaining_maint_dt_ser_list[itr]

    # Need to get new runrate every month
    monthly_tech_rr = wo_tech_mnthly_rr_less_ss.loc[dt_col]

    tech_cnt_by_cap_ser_c = qtrly_tech_cap[dt_col]

    current_month_local_tech_capacity_ser = (tech_cnt_by_cap_ser_c * monthly_tech_rr) - local_tech_supply.loc[:, dt_col]
    current_month_local_tech_capacity_ser.fillna(0, inplace=True) # append this

    check_df = pd.concat([remaining_maint_dt_ser.fillna(0),current_month_local_tech_capacity_ser], axis=1)
    check_df.fillna(0, inplace=True)
    check_df['ser_b3'] = np.where(check_df.iloc[:, 0] >= check_df.iloc[:, 1],check_df.iloc[:, 1], check_df.iloc[:, 0])
    
    current_month_local_tech_supply_ser = check_df.ser_b3
    current_month_local_tech_supply_ser.rename(dt_col, inplace=True)

    local_tech_maint_supply_list.append(current_month_local_tech_supply_ser)

    travel_tech_rem_monthly_supply_value = travel_tech_supply.loc[:,dt_col].apply(lambda x: math.ceil(x / wo_tech_mnthly_rr_less_ss.loc[dt_col])).sum()

    cur_mon_excess_tech_needs_ser, _ = techs_needed_to_complete_work(remaining_maint_dt_ser, current_month_local_tech_supply_ser, monthly_tech_rr
                                                                     , itr, travel_tech_rem_monthly_supply_value)
    current_month_travel_tech_supply_ser = constrain_tech_needs_maint(cur_mon_excess_tech_needs_ser, travel_tech_rem_monthly_supply_value, monthly_tech_rr, dt_col, itr)
    travel_tech_maint_supply_list.append(current_month_travel_tech_supply_ser)

    current_month_internal_tech_supply_ser = current_month_local_tech_supply_ser + current_month_travel_tech_supply_ser
    internal_maint_tech_supply_list.append(current_month_internal_tech_supply_ser)
    

    # TO-DO: rename variables used in the functions below
    w384_ser = get_w384(remaining_maint_dt_ser, current_month_local_tech_supply_ser, current_month_travel_tech_supply_ser,itr)
    b384_ser = get_b384(w384_ser, itr, dt_col)
    
    # Vendor stuff below
    remaining_monthly_maint_budget_val = 0
    cur_mon_excess_external_tech_needs_ser, _ = techs_needed_to_complete_work(remaining_maint_dt_ser, current_month_internal_tech_supply_ser, monthly_tech_rr, itr)
    external_maint_tech_needs_list.append(cur_mon_excess_external_tech_needs_ser)

    dish_date_vendor_cohort_st = format_vendor_cohort_df(external_tech_supply_df, dish_vendor_cohort_st, dt_col)

    nsa_current_external_maint_tech_supply_ser = constrain_external_tech_needs(cur_mon_excess_external_tech_needs_ser, nsa_vendor_cohort_st,'NSA cohort' 
                                                                               , monthly_tech_rr, dt_col, remaining_monthly_maint_budget_val)
    nsa_external_maint_tech_supply_list.append(nsa_current_external_maint_tech_supply_ser)

    # Update value
    remaining_monthly_maint_budget_val = nsa_current_external_maint_tech_supply_ser[dt_col].sum()

    # Apply nsa supply to remaing dt
    current_int_nsa_maint_tech_supply_ser = current_month_internal_tech_supply_ser + nsa_current_external_maint_tech_supply_ser[dt_col]
    cur_mon_excess_post_nsa_tech_needs_ser, _ = techs_needed_to_complete_work(remaining_maint_dt_ser, current_int_nsa_maint_tech_supply_ser, monthly_tech_rr, itr)
    external_maint_post_nsa_tech_needs_list.append(cur_mon_excess_post_nsa_tech_needs_ser)

    dish_current_external_maint_tech_supply_ser = constrain_external_tech_needs(cur_mon_excess_post_nsa_tech_needs_ser, dish_date_vendor_cohort_st,'Dish cohort' 
                                                                                , monthly_tech_rr, dt_col, remaining_monthly_maint_budget_val)
    dish_external_maint_tech_supply_list.append(dish_current_external_maint_tech_supply_ser)

    # Combine external supplies
    current_external_maint_tech_supply_ser = nsa_current_external_maint_tech_supply_ser + dish_current_external_maint_tech_supply_ser
    external_maint_tech_supply_list.append(current_external_maint_tech_supply_ser)

    surplus_maint_dt_ser = remaining_maint_dt_ser - (current_month_internal_tech_supply_ser + current_external_maint_tech_supply_ser[dt_col]) + maint_dt_unconstrained.iloc[:,itr]
    surplus_maint_dt_ser.rename(dt_col, inplace=True)
    remaining_maint_dt_ser_list.append(surplus_maint_dt_ser)    

    if itr==0:
        as_ser = current_month_local_tech_supply_ser + current_month_travel_tech_supply_ser + b384_ser
    else:
        as_ser = current_month_local_tech_supply_ser + current_month_travel_tech_supply_ser + b384_ser + as_maint_list[itr-1]
    
    as_ser.rename(dt_col, inplace=True)
    as_maint_list.append(as_ser)


local_tech_maint_supply = pd.concat(local_tech_maint_supply_list, axis=1)
travel_tech_maint_supply = pd.concat(travel_tech_maint_supply_list, axis=1)
maint_df = pd.concat(remaining_maint_dt_ser_list, axis=1)
met_maint_df = pd.concat(as_maint_list, axis=1)

internal_maint_tech_supply_df = pd.concat(internal_maint_tech_supply_list, axis=1)
external_maint_tech_supply_df = pd.concat(external_maint_tech_supply_list, axis=1)
nsa_external_maint_tech_supply_df = pd.concat(nsa_external_maint_tech_supply_list, axis=1)
dish_external_maint_tech_supply_df = pd.concat(dish_external_maint_tech_supply_list, axis=1)
external_maint_tech_needs_df = pd.concat(external_maint_tech_needs_list, axis=1)
external_maint_post_nsa_tech_needs_df = pd.concat(external_maint_post_nsa_tech_needs_list, axis=1)

### Install Demand + Rolling

#### unconst install + rolling

In [169]:
install_dt_unconstrained_w_backlog['2024-08-01']

Service Territory
Alabama - South        0.0
Alabama Central       24.0
Albany, GA            48.0
Albany, NY            40.0
Albuquerque, NM       12.0
                     ...  
Washington D.C.       38.0
West - Unassigned      0.0
West LA, CA          238.0
Wichita, KS            0.0
Wyoming               11.0
Name: 2024-08-01, Length: 91, dtype: float64

In [170]:
roll_install_dt_unconstrained.head()

install_dt_unconstrained_ = install_dt_unconstrained.copy()
install_dt_unconstrained_['2024-08-01'] = install_dt_unconstrained_['2024-08-01'] + install_dt_unconstrained_w_backlog['2024-07-01']
install_dt_unconstrained_.head()

# install_dt_unconstrained_w_backlog.head()

,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01
Service Territory,,,,,,,,,,,,
Alabama - South,4.0,0.0,14.0,13.0,12.0,14.0,15.0,17.0,18.0,16.0,13.0,12.0
Alabama Central,186.0,6.0,27.0,27.0,27.0,27.0,30.0,34.0,36.0,32.0,27.0,24.0
"Albany, GA",130.0,4.0,56.0,55.0,51.0,57.0,62.0,71.0,75.0,65.0,55.0,49.0
"Albany, NY",144.0,0.0,77.0,76.0,69.0,77.0,85.0,97.0,103.0,90.0,76.0,67.0
"Albuquerque, NM",20.0,0.0,63.0,62.0,56.0,63.0,69.0,79.0,83.0,73.0,62.0,55.0


#### Internal Tech Count

In [171]:
qtrly_tech_cap.sum()

2024-08-01    129.0
2024-09-01    129.0
2024-10-01    135.0
2024-11-01    135.0
2024-12-01    135.0
2025-01-01    149.0
2025-02-01    149.0
2025-03-01    163.0
2025-04-01    163.0
2025-05-01    177.0
2025-06-01    177.0
2025-07-01    177.0
dtype: float64

### Install Supply

In [172]:
# supply internal
internal_tech_supply_df.index.name='Service Territory'

roll_internal_tech_supply_df = get_rolling_df(internal_tech_supply_df)
roll_internal_tech_supply_df.head()

internal_tech_supply_df.head()

# supply external
roll_external_tech_supply_df = get_rolling_df(external_tech_supply_df)

external_tech_supply_df.head()
roll_external_tech_supply_df.head()

# supply local tech
roll_local_tech_supply = get_rolling_df(local_tech_supply)
roll_local_tech_supply.head()

local_tech_supply.head()

# supply travel tech
roll_travel_tech_supply = get_rolling_df(travel_tech_supply)
roll_travel_tech_supply.head()

travel_tech_supply.head()

,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01
Service Territory,,,,,,,,,,,,
Alabama - South,0,0,0,0,0,0,0,0,0,0,0,0
Alabama Central,106,0,0,0,0,0,0,0,0,0,0,0
"Albany, GA",0,0,0,0,0,0,0,0,0,0,0,0
"Albany, NY",48,0,0,0,0,12,24,26,35,42,32,16
"Albuquerque, NM",0,12,0,64,61,57,61,71,78,84,71,65


### Install Walk

In [173]:
install_dt_unconstrained_w_backlog.sum()

2024-07-01    4553.0
2024-08-01    2609.0
2024-09-01     211.0
2024-10-01    4466.0
2024-11-01    4346.0
2024-12-01    3966.0
2025-01-01    4404.0
2025-02-01    4840.0
2025-03-01    5506.0
2025-04-01    5826.0
2025-05-01    5084.0
2025-06-01    4311.0
2025-07-01    3824.0
dtype: float64

In [174]:
local_tech_supply.sum()

2024-08-01    3574.0
2024-09-01    2326.0
2024-10-01     200.0
2024-11-01    3329.0
2024-12-01    3252.0
2025-01-01    3291.0
2025-02-01    3593.0
2025-03-01    4114.0
2025-04-01    4586.0
2025-05-01    4885.0
2025-06-01    4304.0
2025-07-01    3691.0
dtype: float64

In [175]:
internal_tech_supply_df.sum()

2024-08-01    4241.0
2024-09-01    2605.0
2024-10-01     224.0
2024-11-01    3999.0
2024-12-01    3922.0
2025-01-01    3954.0
2025-02-01    4258.0
2025-03-01    4819.0
2025-04-01    5297.0
2025-05-01    5596.0
2025-06-01    5011.0
2025-07-01    4311.0
dtype: float64

In [176]:
external_tech_supply_df.sum()

2024-08-01    256.0
2024-09-01      0.0
2024-10-01      0.0
2024-11-01    400.0
2024-12-01    400.0
2025-01-01     87.0
2025-02-01    147.0
2025-03-01     24.0
2025-04-01    210.0
2025-05-01    233.0
2025-06-01     64.0
2025-07-01      0.0
dtype: float64

#### Unmet Demand MoM

In [177]:
intall_df.sum()

2024-07-01    4553.0
2024-08-01    2665.0
2024-09-01     271.0
2024-10-01    4513.0
2024-11-01    4460.0
2024-12-01    4104.0
2025-01-01    4467.0
2025-02-01    4902.0
2025-03-01    5565.0
2025-04-01    5884.0
2025-05-01    5139.0
2025-06-01    4375.0
2025-07-01    3888.0
dtype: float64

#### Rolling Unmet Demand MoM

In [178]:
roll_install_delta_dt_v_int_ext = roll_install_dt_unconstrained - (roll_internal_tech_supply_df + roll_external_tech_supply_df)
roll_install_delta_dt_v_int_ext.sum()

2024-08-01    2665.0
2024-09-01     271.0
2024-10-01    4513.0
2024-11-01    4460.0
2024-12-01    4104.0
2025-01-01    4467.0
2025-02-01    4902.0
2025-03-01    5565.0
2025-04-01    5884.0
2025-05-01    5139.0
2025-06-01    4375.0
2025-07-01    3888.0
dtype: float64

### Maint Demand + Rolling

#### unconst maint + rolling

In [179]:
roll_maint_dt_unconstrained = get_rolling_df(maint_dt_unconstrained_w_backlog).iloc[:, 1:]
roll_maint_dt_unconstrained.head()

# maint_dt_unconstrained.head()
maint_dt_unconstrained_w_backlog.head()

,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01
Service Territory,,,,,,,,,,,,,
Alabama - South,32,32,32,32,34,35,36,38,39,41,43,44,46
Alabama Central,46,63,65,66,68,71,74,77,80,83,87,90,92
"Albany, GA",91,99,104,105,110,116,121,126,133,140,147,154,159
"Albany, NY",39,50,54,54,62,69,76,84,92,102,112,121,129
"Albuquerque, NM",23,24,25,25,32,38,43,49,57,64,73,80,86


### Maint Supply

In [180]:
roll_internal_maint_tech_supply_df = get_rolling_df(internal_maint_tech_supply_df)
roll_internal_maint_tech_supply_df.sum()

roll_external_maint_tech_supply_df = get_rolling_df(external_maint_tech_supply_df)
roll_external_maint_tech_supply_df.sum()

roll_maint_dt_unconstrained.sum()

2024-08-01     16932
2024-09-01     25900
2024-10-01     34889
2024-11-01     44319
2024-12-01     54183
2025-01-01     64444
2025-02-01     75146
2025-03-01     86336
2025-04-01     98079
2025-05-01    110405
2025-06-01    123242
2025-07-01    136506
dtype: int64

### Maint. walk

In [181]:
maint_dt_unconstrained_w_backlog.sum()

2024-07-01     8222
2024-08-01     8710
2024-09-01     8968
2024-10-01     8989
2024-11-01     9430
2024-12-01     9864
2025-01-01    10261
2025-02-01    10702
2025-03-01    11190
2025-04-01    11743
2025-05-01    12326
2025-06-01    12837
2025-07-01    13264
dtype: int64

In [182]:
local_tech_maint_supply.sum()

2024-08-01    3747.0
2024-09-01    4774.0
2024-10-01    6806.0
2024-11-01    4310.0
2024-12-01    4446.0
2025-01-01    5282.0
2025-02-01    5085.0
2025-03-01    6029.0
2025-04-01    5708.0
2025-05-01    6320.0
2025-06-01    6848.0
2025-07-01    7418.0
dtype: float64

In [183]:
internal_maint_tech_supply_df.sum()

2024-08-01    4540.0
2024-09-01    5993.0
2024-10-01    7171.0
2024-11-01    5224.0
2024-12-01    5299.0
2025-01-01    6623.0
2025-02-01    6060.0
2025-03-01    7653.0
2025-04-01    6812.0
2025-05-01    7359.0
2025-06-01    8277.0
2025-07-01    9367.0
dtype: float64

In [184]:
external_maint_tech_supply_df.sum()

2024-08-01    1000.0
2024-09-01    1000.0
2024-10-01    1000.0
2024-11-01    1000.0
2024-12-01    1000.0
2025-01-01    1000.0
2025-02-01    1000.0
2025-03-01    1000.0
2025-04-01    1000.0
2025-05-01    1000.0
2025-06-01    1000.0
2025-07-01    1000.0
dtype: float64

#### Unmet Maint Demand

In [185]:
maint_df.sum()

2024-07-01     8222.0
2024-08-01    11392.0
2024-09-01    13367.0
2024-10-01    14185.0
2024-11-01    17391.0
2024-12-01    20956.0
2025-01-01    23594.0
2025-02-01    27236.0
2025-03-01    29773.0
2025-04-01    33704.0
2025-05-01    37671.0
2025-06-01    41231.0
2025-07-01    44128.0
dtype: float64

#### Rolling Unmet Maint Demand MoM

In [186]:
roll_maint_delta_dt_v_int_ext = roll_maint_dt_unconstrained - (roll_internal_maint_tech_supply_df + roll_external_maint_tech_supply_df)
roll_maint_delta_dt_v_int_ext.sum()

2024-08-01    11392.0
2024-09-01    13367.0
2024-10-01    14185.0
2024-11-01    17391.0
2024-12-01    20956.0
2025-01-01    23594.0
2025-02-01    27236.0
2025-03-01    29773.0
2025-04-01    33704.0
2025-05-01    37671.0
2025-06-01    41231.0
2025-07-01    44128.0
dtype: float64

### Write to excel

In [187]:
# intall_df.index.name = 'Service Territory'
# test_df = intall_df.reset_index()
# long_install_df = pd.melt(test_df, id_vars='Service Territory', value_vars=intall_df.columns)
# long_install_df.set_index('Service Territory', inplace=True)
# long_install_df

In [188]:
# book = load_workbook('Ingest Bottoms Up_ Install vs Maint Forecast - Install __ Maint Aged Forecast - 5.2024 copy.xlsx')
writer = pd.ExcelWriter('tech_hiring_model_output.xlsx', engine='openpyxl') 
# writer.book = book

cap_perc_ma3.to_excel(writer, sheet_name='cap_perc_ma3')
cap_perc.to_excel(writer, sheet_name='cap_perc')

wo_tech_mnthly_rr_less_ss.to_excel(writer, sheet_name='monthly_run_rate')
qtrly_tech_cap.to_excel(writer, sheet_name='tech_count')

install_dt_unconstrained_.to_excel(writer, sheet_name='install_dt')
maint_dt_unconstrained_.to_excel(writer, sheet_name='maint_dt')

local_tech_supply.to_excel(writer, sheet_name='local_tech_supply')
travel_tech_supply.to_excel(writer, sheet_name='travel_tech_supply')

local_tech_maint_supply.to_excel(writer, sheet_name='local_tech_maint_supply')
travel_tech_maint_supply.to_excel(writer, sheet_name='travel_tech_maint_supply')

internal_tech_supply_df.to_excel(writer, sheet_name='internal_tech_supply')
external_tech_supply_df.to_excel(writer, sheet_name='external_tech_supply')

internal_maint_tech_supply_df.to_excel(writer, sheet_name='internal_maint_tech_supply')
external_maint_tech_supply_df.to_excel(writer, sheet_name='external_maint_tech_supply')

nsa_external_maint_tech_supply_df.to_excel(writer, sheet_name='nsa_external_maint_tech_supply')
dish_external_maint_tech_supply_df.to_excel(writer, sheet_name='dish_external_maint_tech_supply')

intall_df.to_excel(writer, sheet_name='unmet_install_dt')
maint_df.to_excel(writer, sheet_name='unmet_maint_df')
# roll_install_delta_dt_v_int_ext.to_excel(writer, sheet_name='roll_install_delta_dt_v_int_ext')


writer.close()